In [ ]:
import json
import statistics
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000
import matplotlib.pyplot as plt
from coordinate_converter import ssb_grid_id_to_utm_centroid, utm_to_latitude_longitude, utm_distance

In [ ]:
od = {}
with open(f'data/od.json', 'r') as r:
    od = json.load(r)

In [ ]:
od_paths = {}
with open(f'data/od_paths.json', 'r') as r:
    od_paths = json.load(r)

In [ ]:
old = []
new = []
diffs = []
coords = []

for start_id in od_paths.keys():
    if start_id == "update_period_minutes":
        continue
    for end_id in od_paths[start_id]:
        try:
            travel_time = od[start_id][end_id]
            path_time = od_paths[start_id][end_id]["travel_time"]/60
            coords.append((start_id, end_id))
            old.append(travel_time)
            new.append(path_time)
            diffs.append(round(path_time-travel_time, 1))
        except:
            continue

In [ ]:
print("Average travel time difference:", statistics.fmean(diffs))
print("Biggest differences:", max(diffs), min(diffs))
print("Median travel time old:", statistics.median(old))
print("Median travel time new:", statistics.median(new))

In [ ]:
size_plt = len(old) # 1_000_000
old_plt = old[:size_plt]
new_plt = new[:size_plt]

old_plt = sorted(old_plt)
new_plt = sorted(new_plt)

plt.scatter(range(size_plt), old_plt, s=1, alpha=1)
plt.scatter(range(size_plt), new_plt, s=1, alpha=1)

In [ ]:
plt.scatter(range(size_plt), sorted(diffs), s=1, alpha=1)
plt.axhline(y=0, color='r', linestyle='-')

In [ ]:
size = 0
processed = []
for i in range(len(diffs)):
    if (-30 > diffs[i] > -50 or 30 < diffs[i] < 50) and coords[i][0][0] != '_' and coords[i][1][0] != '_':
        size += 1
        start = ssb_grid_id_to_utm_centroid(int(coords[i][0]))
        end = ssb_grid_id_to_utm_centroid(int(coords[i][1]))
        newPath = True
        for a, b in processed:
            start_dist = utm_distance(start, a)
            end_dist = utm_distance(end, b)
            if start_dist + end_dist < 20000:
                newPath = False
                continue
        if newPath:
            processed.append((start, end))
            print("old", round(old[i]), "new", round(new[i]), utm_to_latitude_longitude(start), utm_to_latitude_longitude(end))

print(size)
print(len(processed))

In [ ]:
google = [90, 82.5, 100, 120, 80, 40, 75, 140, 45, 90, 110, 90, 82.5, 130, 45, 90, 92.5, 100, 77.5, 100, 0, 100, 50, 120]
old_select = [142, 128, 154, 82, 66, 104, 159, 106, 32, 131, 79, 128, 121, 81, 32, 64, 60, 49, 37, 60, 34, 61, 6, 88]
new_select = [88, 77, 99, 133, 118, 31, 77, 158, 62, 92, 112, 83, 76, 120, 63, 95, 91, 83, 68, 95, 0, 98, 49, 119]

old_select = [x for _, x in sorted(zip(google, old_select))]
new_select = [x for _, x in sorted(zip(google, new_select))]
google = sorted(google)

x = range(len(google))
plt.plot(x, google, c="green")
plt.plot(x, old_select, c="red")
plt.plot(x, new_select, c="blue")